### Produce ChemBERTa embeddings for Drugs

#### Import libraries and BioBERT model

In [1]:
!pip install pubchempy
!pip install rdkit
!pip install progressbar2

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import torch
import pickle
import json
import numpy as np

import pubchempy as pc
from rdkit import Chem
from rdkit.Chem import AllChem
from progressbar import ProgressBar

from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('seyonec/PubChem10M_SMILES_BPE_450k')
tokenizer = AutoTokenizer.from_pretrained('seyonec/PubChem10M_SMILES_BPE_450k')


C:\Users\dlezo\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


#### Load the Drugs SMILES from DrugBank

In [3]:
with open('./Data/TWOSIDES/TWOSIDES-DB-SMILES.json', 'r') as file:
    db_to_smiles = json.load(file)
# add some extra SMILES
db_smiles_df = pd.read_csv('./Data/TWOSIDES/PubChem-DrugBank-Name-SMILES.csv')
for _, row in db_smiles_df.iterrows():
    db_to_smiles[row['DrugBankID']] = row['SMILES']
with open('./Data/TWOSIDES/rx_norm_to_db.pkl', 'rb') as file:
    rx_norm_to_db = pickle.load(file)

#### Get the embedding from ChemBERTa

In [4]:
embeddings = {}
i = 0
step = len(rx_norm_to_db.keys())//100
bar = ProgressBar(max_value=100)
for rx_norm_id in rx_norm_to_db.keys():
    if i%step==0:
        bar.next()
    encoded_input = tokenizer(db_to_smiles[rx_norm_to_db[rx_norm_id]], return_tensors='pt')
    output_embedding = model(**encoded_input)['last_hidden_state'][:,0,:]
    embeddings[rx_norm_id] = output_embedding
    i += 1
bar.finish()

100% (100 of 100) |######################| Elapsed Time: 0:01:50 Time:  0:01:500109


In [6]:
# Get the embeddings in a list
embeddings768 = []
for rx_norm_id in rx_norm_to_db.keys():
    embeddings768.append(embeddings[rx_norm_id].detach().numpy()[0])
embeddings768 = np.array(embeddings768)

#### Reduce the embeddings dimensionality with PCA

In [7]:
import numpy as np
from sklearn.decomposition import PCA

size = 128
pca = PCA(n_components=size, svd_solver='full')
embeddings_pca = pca.fit_transform(embeddings768)

print('Number of components:', pca.n_components_)
print('Covariance covered:', pca.explained_variance_ratio_.sum())

Number of components: 128
Covariance covered: 0.92626125


In [8]:
embeddings_size_pca = {}
i = 0
for rx_norm_id in embeddings.keys():
    embeddings_size_pca[rx_norm_id] = embeddings_pca[i]
    i += 1

In [9]:
with open('./Data/Embeddings/chemberta_embeddings_'+str(size)+'_pca.pkl', 'wb') as file:
    pickle.dump(embeddings_size_pca, file)
    print(file.name)


./Data/TWOSIDES-chemberta_embeddings_128_pca.pkl
